In [73]:
from sqlalchemy import *
engine = create_engine('sqlite:///onto.db', echo = False)
c = engine.connect()

In [164]:
c = engine.connect()
c.execution_options(autocommit=True).execute("DELETE FROM final_node_decisions;")
# results.fetchall()

2020-12-13 02:20:25,575 INFO sqlalchemy.engine.base.Engine DELETE FROM final_node_decisions;


INFO:sqlalchemy.engine.base.Engine:DELETE FROM final_node_decisions;


2020-12-13 02:20:25,583 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-13 02:20:25,587 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [142]:
engine = create_engine('sqlite:///onto.db', echo = True)
    # conn = sqlite3.connect('onto.db')
    # c = conn.cursor()
c = engine.connect()
trans = c.begin()
query = """SELECT * FROM node_decisions INNER JOIN nodes ON node_decisions.node_id =nodes.id """
result = c.execute(query)
result.fetchall()

2020-12-13 01:11:19,622 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2020-12-13 01:11:19,632 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-13 01:11:19,640 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

2020-12-13 01:11:19,666 INFO sqlalchemy.engine.base.Engine SELECT * FROM node_decisions INNER JOIN nodes ON node_decisions.node_id =nodes.id 


INFO:sqlalchemy.engine.base.Engine:SELECT * FROM node_decisions INNER JOIN nodes ON node_decisions.node_id =nodes.id 


2020-12-13 01:11:19,683 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


[(1, 4, 1, 1324455346191020032, 4, 1, "https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Domino'SPizza"),
 (2, 3, 0, 1324455346191020032, 3, 1, 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Peri-Peri'),
 (3, 2, 1, 1324455346191020032, 2, 1, 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Marinara'),
 (4, 1, 1, 1324455346191020032, 1, 1, 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#TandooriPizza'),
 (5, 5, 1, 1324455346191020032, 5, 1, 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#IndianPizza'),
 (6, 5, 1, 1324449088826343424, 5, 1, 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#IndianPizza'),
 (7, 1, 1, 1324449088826343424, 1, 1, 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#TandooriPizza'),
 (8, 3, 1, 1324449088826343424, 3, 1, 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Peri-Peri'),
 (9, 2, 1, 1324449088826343424, 2, 1, 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Marinara'),
 (10, 4, 1, 1324449088826343424, 4, 1, "h

In [160]:
engine = create_engine('sqlite:///onto.db', echo = True)
    # conn = sqlite3.connect('onto.db')
    # c = conn.cursor()
c = engine.connect()
trans = c.begin()
query = """SELECT domain, range, property FROM class_relations 
        INNER JOIN final_class_decisions ON class_relations.id = final_class_decisions.relation_id
        INNER JOIN ontologies ON class_relations.onto_id = ontologies.id
        WHERE ontologies.name = :name AND final_class_decisions.approved != 0"""
result = c.execute(query, {'name': 'pizza'})
result.fetchall()

2020-12-13 02:14:12,230 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2020-12-13 02:14:12,233 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-13 02:14:12,241 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

2020-12-13 02:14:12,256 INFO sqlalchemy.engine.base.Engine SELECT domain, range, property FROM class_relations 
        INNER JOIN final_class_decisions ON class_relations.id = final_class_decisions.relation_id
        INNER JOIN ontologies ON class_relations.onto_id = ontologies.id
        WHERE ontologies.name = :name AND final_class_decisions.approved != 0


INFO:sqlalchemy.engine.base.Engine:SELECT domain, range, property FROM class_relations 
        INNER JOIN final_class_decisions ON class_relations.id = final_class_decisions.relation_id
        INNER JOIN ontologies ON class_relations.onto_id = ontologies.id
        WHERE ontologies.name = :name AND final_class_decisions.approved != 0


2020-12-13 02:14:12,261 INFO sqlalchemy.engine.base.Engine {'name': 'pizza'}


INFO:sqlalchemy.engine.base.Engine:{'name': 'pizza'}


[('https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Pizza', "https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Domino'SPizza", 'hasInstance'),
 ('https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Peri-Peri', 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Pizza', 'subclassOf'),
 ('https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Marinara', 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Pizza', 'isToppingOf'),
 ('https://serc.iiit.ac.in/downloads/ontology/pizza.owl#TandooriPizza', 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#IndianPizza', 'subclassOf')]

In [31]:
from owlready2 import *

from re import finditer
def split_by_camel_case(identifier):
    # Split string by camel-case
    matches = finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return " ".join([m.group(0) for m in matches])

ont = get_ontology("data/input/ontologies/pizza.owl").load()
ont_classes = list(ont.classes())
classes = [split_by_camel_case(elem.label.first()).lower() for elem in ont_classes]

##if term1 not in set(classes) or term2 in set(classes):
    ##append it
# idx1 = classes.index(term1) 
# idx2 = classes.index(term2)
# ont_classes[idx1], ont_classes[idx2]

In [121]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next
class Solution:
    def getNum(self, node):
        l = []
        currNode = node
        while currNode:
            l.append(str(currNode.val))
            currNode = currNode.next
        num = int(''.join(l[::-1]))
        return num
        
    def addTwoNumbers(self, l1: ListNode, l2: ListNode) -> ListNode:
        n1 = self.getNum(l1)
        n2 = self.getNum(l2)
        print (n1, n2)
        added = str(n1 + n2)
        finalnodes = []
        for intg in added[::-1]:
            finalnodes.append(ListNode(intg))
        for i,f in enumerate(finalnodes[:-1]):
            f.next = finalnodes[i+1]
        finalnodes[-1].next = None
        return finalnodes[0]
        


In [23]:
#F3
import tensorflow_hub as hub
from scipy import spatial
# USE_link = "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed"
# model = hub.load(USE_link)

def extractUSEEmbeddings(words):
    word_embeddings = model(words)
    return word_embeddings.numpy()

def cos_sim(a,b):
    # Returns cosine similarity of two vectors
    return 1 - spatial.distance.cosine(a, b)

def generateScore(text_array):
    all_embs = extractUSEEmbeddings(text_array + ["Pizza"])
    return [cos_sim(tweet_emb, all_embs[-1]) for tweet_emb in all_embs[:-1]]
print (cos_sim(*extractUSEEmbeddings(["Season's eatings! Driving around to look at holiday lights? Drive up to one of our pizzerias for curbside pick-up and chow down on your favorite deep dish from Lou's SnowmanSnowflake", "Information Security"])))
print (cos_sim(*extractUSEEmbeddings(["Football Sunday calls for deep dish! American football. No matter who you're rooting for, deep dish is something we call can agree on! Slice of pizza", "Information Security"])))
print (cos_sim(*extractUSEEmbeddings(["Today's the perfect day to get your craft on and build a gingerbread house! Check out this gingerbread Lou Malnati's made by our friend @carlybreakstone96. Craving Lou's too? Order some deep dish and eat while you build!", "Information Security"])))
print (cos_sim(*extractUSEEmbeddings(["Psst... Christmas is in 1 week. Still shopping for people on your list? Spread some hoLOUday cheer with a cheesy stocking stuffer! Give the gift of a Lou's gift card this holiday season & get a FREE bonus certificate, (you've earned it).", "Information Security"])))
print (cos_sim(*extractUSEEmbeddings(["LAST CHANCE to ship deep dish for standard delivery before Christmas!Place your Tastes of Chicago order by TOMORROW, Friday, December 18th for standard delivery before Christmas with no extra shipping fees.", "Information Security"])))


-0.005545307882130146
0.025124356150627136
0.017693020403385162
-0.009036985225975513
-0.010162311606109142


In [24]:
#F3
import tensorflow_hub as hub
from scipy import spatial
# USE_link = "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed"
# model = hub.load(USE_link)

def extractUSEEmbeddings(words):
    word_embeddings = model(words)
    return word_embeddings.numpy()

def cos_sim(a,b):
    # Returns cosine similarity of two vectors
    return 1 - spatial.distance.cosine(a, b)

def generateScore(text_array):
    all_embs = extractUSEEmbeddings(text_array + ["Pizza"])
    return [cos_sim(tweet_emb, all_embs[-1]) for tweet_emb in all_embs[:-1]]
print (cos_sim(*extractUSEEmbeddings(["When it’s the #FirstDayOfWinter and you wanna warm up with as many pizza nights as possible. How many times would you press this button? #JetsPizza", "Information Security"])))
print (cos_sim(*extractUSEEmbeddings(["There are five differences between these two pics. Go ahead, try and spot ‘em.", "Information Security"])))
print (cos_sim(*extractUSEEmbeddings(["A perfect Saturday night looks like this. #JetsPizza", "Information Security"])))
print (cos_sim(*extractUSEEmbeddings(["When you realize grilled cheese is out and Jet’s Bread is IN. Pair your tomato soup with Jet’s Bread. #JetsPizza ", "Information Security"])))
print (cos_sim(*extractUSEEmbeddings(["What would you pair with your Detroit-Style pizza?", "Information Security"])))


0.009363527409732342
-0.0381292924284935
-0.0821034163236618
0.020301612094044685
-0.070344477891922


In [25]:
#F3
import tensorflow_hub as hub
from scipy import spatial
# USE_link = "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed"
# model = hub.load(USE_link)

def extractUSEEmbeddings(words):
    word_embeddings = model(words)
    return word_embeddings.numpy()

def cos_sim(a,b):
    # Returns cosine similarity of two vectors
    return 1 - spatial.distance.cosine(a, b)

def generateScore(text_array):
    all_embs = extractUSEEmbeddings(text_array + ["Pizza"])
    return [cos_sim(tweet_emb, all_embs[-1]) for tweet_emb in all_embs[:-1]]
print (cos_sim(*extractUSEEmbeddings(["Game recognize game.", "Information Security"])))
print (cos_sim(*extractUSEEmbeddings(["What a busy few hours for Santa and Mrs. Claus. Now time to get back to wrapping gifts. Thank you to the managers of Santa's schedule!", "Information Security"])))
print (cos_sim(*extractUSEEmbeddings(["Rehaj, 6, from Moose Jaw, SK. asked: Why does Santa always wear red, and can you come more than once a year?", "Information Security"])))
print (cos_sim(*extractUSEEmbeddings(["Bryson & Easton, 7&4, from Thunder Bay, ON. asked: Does Mrs. Claus help with planning? And why is Rudolph's nose red?", "Information Security"])))
print (cos_sim(*extractUSEEmbeddings(["Roman, 6, from Barrie, ON. asked a great one: Are the Santa's at the mall you, or are they helpers dressed like you?", "Information Security"])))


0.09831051528453827
-0.027837885543704033
0.03359723091125488
-0.007910827174782753
-0.04666958749294281


In [18]:
import numpy as np

class Solution:
    
    def sum_mat(self, mat, indices):
        final_indices = []
        for i,index in enumerate(indices):
            if index < 0:
                index = 0
            if i == 0 and index >= self.m:
                return 0
            if i == 1 and index <= 0:
                return 0
            if i == 2 and index >= self.n:
                return 0
            if i == 3 and index <= 0:
                return 0
            
            if i>=2 and index >= self.n:
                index = self.n if i%2 == 1 else (self.n - 1) 
            elif i<2 and index >= self.m:
                index = self.m if i%2 == 1 else (self.m - 1) 
            final_indices.append(index)
        a,b,c,d = final_indices
        print (a,b,c,d)
        ans = np.sum(mat[a:b, c:d])
        print ("Ans,", ans)
        return ans
        
    def calculate_ans(self, mat, i, j):
        if self.memo[i][j]!=0:
            return self.memo[i][j]
        print (i,j, self.m)
        if (i+1) < self.m and self.memo[i+1][j]!=0:
            print(1)
            return self.memo[i+1][j] + self.sum_mat(mat, [i-self.k, i-self.k+1, j-self.k, j+self.k+1]) - self.sum_mat(mat, [i+self.k+1, i+self.k+2, j-self.k, j+self.k+1])
        
        elif (i-1) >= 0 and self.memo[i-1][j]!=0:
            print(2)
            return self.memo[i-1][j] + self.sum_mat(mat, [i+self.k, i+self.k+1, j-self.k, j+self.k+1]) - self.sum_mat(mat, [i-self.k-1, i-self.k, j-self.k, j+self.k+1])
        
        elif (j+1) < self.n and self.memo[i][j+1] != 0:
            print(3)
            return self.memo[i][j+1] + self.sum_mat(mat, [i-self.k, i+self.k+1, j-self.k, j-self.k+1]) - self.sum_mat(mat, [i-self.k, i+self.k+1, j+self.k+1, j+self.k+2])
        
        elif (j-1) >= 0 and self.memo[i][j-1]!=0:
            print(4)
            return self.memo[i][j-1] - self.sum_mat(mat, [i-self.k, i+self.k+1, j-self.k-1, j-self.k]) + self.sum_mat(mat, [i-self.k, i+self.k+1, j+self.k, j+self.k+1])
        
        
        return self.sum_mat(mat, [i-self.k, i+self.k+1, j-self.k, j+self.k+1])
        
    def matrixBlockSum(self, mat, K):
        self.m, self.n = np.array(mat).shape
        self.k = K
        self.memo = [[0 for i in range(self.n)] for j in range(self.m)]
        for i in range(self.m):
            for j in range(self.n):
                print (i,j)
                self.memo[i][j] = self.calculate_ans(np.array(mat), i, j)
        return self.memo
    
s = Solution()
s.matrixBlockSum([[1,2,3],[4,5,6],[7,8,9]], 1)

0 0
0 0 3
0 2 0 2
Ans, 12
0 1
0 1 3
4
0 2 2 3
Ans, 9
0 2
0 2 3
4
0 2 0 1
Ans, 5
1 0
1 0 3
2
2 3 0 2
Ans, 15
1 1
1 1 3
2
2 3 0 3
Ans, 24
1 2
1 2 3
2
2 3 1 3
Ans, 17
2 0
2 0 3
2
0 1 0 2
Ans, 3
2 1
2 1 3
2
0 1 0 3
Ans, 6
2 2
2 2 3
2
0 1 1 3
Ans, 5


[[12, 21, 16], [27, 45, 33], [24, 39, 28]]

In [19]:
a = [1,2,3]
a[:0]

[]